# process_cmip6_files.ipynb
Post-processing of CMIP6 data that have been downloaded from ESGF:
1. Regrid to regular lon/lat grid (using CDO) and calculate statistics (e.g. CDO's yearmean).

B. S. Grandey, 2022

In [1]:
! date

Fri Jan 28 20:41:18 +08 2022


In [2]:
import contextlib
import pathlib
from cdo import Cdo, CDOException

In [3]:
cdo = Cdo()

print(f'CDO version: {cdo.version()}')
print(f'cdo.py bindings version: {cdo.__version__()}')

CDO version: 2.0.3
cdo.py bindings version: 1.5.4


## Base paths

In [4]:
# CMIP6 input data downloaded from ESGF (organised as <variable>/<source_id>_<member_id>/<instance_id>)
in_base = pathlib.Path('~/Data/p22b/CMIP6/').expanduser()
# Output data
out_base = pathlib.Path('~/Data/p22c/CMIP6/').expanduser()
out_base.mkdir(exist_ok=True)

## Function to regrid available data (using CDO) for a given variable, table_id, source-member pair, and experiment
Notes regarding available interpolation methods, after testing CDO's gencon2, gencon, and genbic on Omon zos data:
1. For unstructured grids, gencon/gencon2 works but not genbic ("Bilinear/bicubic interpolation doesn't support unstructured source grids!").
2. For curvilinear grids, genbic works. gencon/gencon2 mostly works, but fails for some grids (with most common error being "Source grid cell corner coordinates missing!").
3. For lonlat grids, all methods work.

Based on the above observations, one possibility would be to try gencon2 (2nd order conservative). If gencon2 fails, then use gencon (1st order conservative). If gencon2 and gencon both fail, then use genbic (bicubic). In case all three fail, include gennn (nearest neighbour) as a final fallback.

In [5]:
def regrid_using_cdo(variable='zos',
                     table_id='Omon',
                     source_member='ACCESS-CM2_r1i1p1f1',
                     experiment='historical',
                     stat_str='-yearmean',  # calculate statistical values after regridding?
                     preferred_methods=['gencon2', 'gencon', 'genbic', 'gennn', None],  # highest priority first
                     force=False,  # overwrite previously produced files?
                    ):
    
    print(f'---- {variable} {table_id} {source_member} {experiment} {stat_str} ----')
    
    # List of pre-existing files skipped (assuming force=False)
    existing_list = []
    # List of output files written
    out_list = []
    
    # Search for available input directories, then identify most appropriate directory
    in_dirs = list(in_base.glob(f'{variable}/{source_member}/CMIP6.*.{experiment}.*.{table_id}.{variable}.*'))
    in_dirs = sorted(in_dirs)
    # Case: no input directories
    if len(in_dirs) == 0:
        print('No suitable input data found')
        return None  # exit (if no input directories)
    # Case: one suitable input directory
    elif len(in_dirs) == 1:
        in_dir = in_dirs[0]
        print(f'1 input dir: {in_dir.name} ({len(list(in_dir.glob("*.nc")))} nc file(s))')
    # Case: more than one suitable input directory
    else:
        print(f'{len(in_dirs)} input dirs:')
        for d in in_dirs:
            print(f'{d.name} ({len(list(d.glob("*.nc")))} nc file(s))')
        # Select most recently dated version
        vmax = 0  # initialise "max" version as 0
        for d in in_dirs:
            vmax = max(vmax, int(d.name[-8:]))
        for d in in_dirs:
            if str(vmax) not in d.name:
                in_dirs.remove(d)
        # After version selection, does only one input dir remain?
        if len(in_dirs) == 1:
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (most recent) ')
        # Prefer data that has already been remapped
        else:
            for d in in_dirs:
                if '.gn.' in d.name:
                    in_dirs.remove(d)
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (non-native grid)')
    
    # Identify input data files
    in_fns = sorted(list(in_dir.glob('*.nc')))
    # Input grid type
    try:
        with contextlib.redirect_stdout(None):  # suppress CDO error message details (if any)
            grid_str = [s for s in cdo.sinfo(input=f'{in_fns[0]}') if 'points' in s][0]
        in_grid_type = grid_str.split(':')[1].strip()
        in_grid_points = grid_str.split(':')[-1].strip()
        print(f'Input data grid type is {in_grid_type} ({in_grid_points})')
    except CDOException:  # if Globus transfer is in progress, incomplete files may be encountered
        print(f'Failed to read {in_fns[0].name} - is it currently being transferred?')
        return None  # exit (if incomplete file encountered)
    # Output data directory
    if stat_str:
        out_dir = out_base.joinpath(f'regrid_{stat_str.strip("-")}/{variable}/{source_member}/{in_dir.name}')
    else:
        out_dir = out_base.joinpath(f'regrid/{variable}/{source_member}/{in_dir.name}')
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # If points=1 (e.g. due to already being a global mean), then no need to regrid
    if in_grid_points == 'points=1':
        print('No need to regrid')
        # Loop over in_fns (if points=1)
        for in_fn in in_fns:
            # Output filename (if points=1)
            if stat_str:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.{stat_str.strip("-")}.nc')
            else:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.1d.nc')
            # Do not overwrite, unless force=True (if points=1)
            if out_fn.exists() and not force:
                existing_list.append(out_fn.name)
                continue
            # Write output file (if points=1)
            cdo.selname(variable, input=f'{stat_str} {in_fn}', output=f'{out_fn}')
            if out_fn.exists():
                out_list.append(out_fn.name)
            else:
                print(f'Failed to write {out_fn.name}')
        # Print summary of existing files skipped (if points=1)
        if len(existing_list) == 1:
            print(f'Skipped existing file: {existing_list[0]}')
        elif len(existing_list) > 1:
            print(f'Skipped {len(existing_list)} existing files, inc {existing_list[0]}')
        # Print summary of files written (if points=1)
        if len(out_list) == 1:
            print(f'Written {out_list[0]}')
        elif len(out_list) > 1:
            print(f'Written {len(out_list)} files, inc {out_list[0]}')
        return out_list  # exit (if points=1)
    
    # Weights data directory
    weights_dir = out_base.joinpath(f'regrid_weights/{variable}/{source_member}/{in_dir.name}')
    weights_dir.mkdir(parents=True, exist_ok=True)
    # Generate regridding weights file
    for method in preferred_methods:
        print(f'Trying {method}')
        weights_fn = weights_dir.joinpath(f'{in_fns[0].stem}.{method}_weights.nc')
        # If all preferred methods have been exhausted, then exit
        if method == None:
            return None  # exit
        # For unstructured grids, do not use gencon2
        if method == 'gencon2' and in_grid_type == 'unstructured':
            print(f'Skipping gencon2 for unstructured grid')
            continue
        # If weights file already exists, use pre-existing file (unless force=True)
        if weights_fn.exists() and not force:
            print(f'{weights_fn.name} already exists')
            break  # if weights file exists, no need to try other methods
        # Otherwise, try calculating weights and writing file
        else:
            try:
                with contextlib.redirect_stdout(None):  # suppress CDO error message details
                    if method == 'gencon2':
                        cdo_res = cdo.gencon2('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gencon':
                        cdo_res = cdo.gencon('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')          
                    elif method == 'genbic':
                        cdo_res = cdo.genbic('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gennn':
                        cdo_res = cdo.gennn('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    else:
                        continue
                if weights_fn.exists():
                    print(f'Written {weights_fn.name}')
                    break  # if file written successfully, then no need to try other methods
            except CDOException:
                print(f'Failed to write {weights_fn.name}')
    
    # Loop over input data files and regrid
    print(f'{len(in_fns)} input data file(s) to regrid')
    for in_fn in in_fns:
        # Output data filename
        if stat_str:
            out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}.{stat_str.strip("-")}.nc')
        else:
            out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}.nc')
        # If output file already exists, do not overwrite (unless force=True)
        if out_fn.exists() and not force:
            existing_list.append(out_fn.name)
            continue
        # Regrid using CDO
        cdo_res = cdo.sellonlatbox(0, 360, -90, 90,  # shift longitudes
                                   input=f'{stat_str} -selname,{variable} -remap,global_1,{weights_fn} {in_fn}',
                                   output=f'{out_fn}')
        # Check if written
        if out_fn.exists():
            out_list.append(out_fn.name)
        else:
            print(f'Failed to write {out_fn.name}')
    
    # Print summary of existing files skipped
    if len(existing_list) == 1:
        print(f'Skipped existing file: {existing_list[0]}')
    elif len(existing_list) > 1:
        print(f'Skipped {len(existing_list)} existing files, inc {existing_list[0]}')
    # Print summary of files written (if points=1)
    if len(out_list) == 1:
        print(f'Written {out_list[0]}')
    elif len(out_list) > 1:
        print(f'Written {len(out_list)} files, inc {out_list[0]}')
    
    return out_list  # exit

## Apply function(s) to variables and experiments of interest

### Annual means (yearmean) of regridded Omon data

In [6]:
%%time
# For variables and experiments of interest, apply regridding function to all available source-member pairs
# Loop over variables of interest
table_id = 'Omon'
for variable in ['zostoga', 'thetaoga',  # 1D ocean
                 'zos', 'hfds']:  # 2D ocean
    # Identify and loop over available source-member pairs
    source_member_pairs = sorted([d.name for d in in_base.glob(f'{variable}/[!.]*_*')])
    for source_member in source_member_pairs:
        # Loop over experiments of interest - include piControl for Omon
        for experiment in ['piControl', 'historical', 'ssp585', 'ssp370', 'ssp245', 'ssp126']:
            # Regrid
            temp = regrid_using_cdo(variable=variable,
                                    table_id=table_id,
                                    source_member=source_member,
                                    experiment=experiment,
                                    stat_str='-yearmean',
                                    force=False)
        print('')  # empty line between source-member pairs

---- zostoga Omon ACCESS-CM2_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.piControl.r1i1p1f1.Omon.zostoga.gn.v20191112 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.yearmean.nc
---- zostoga Omon ACCESS-CM2_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r1i1p1f1.Omon.zostoga.gn.v20191108 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.yearmean.nc
---- zostoga Omon ACCESS-CM2_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp585.r1i1p1f1.Omon.zostoga.gn.v20210317 (2 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped 2 existing files, inc zostoga_Omon_ACCESS-CM2_ssp585_r1i1p1f1_gn_201501-210012.yearmean.nc
---- zostog

Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_CMCC-ESM2_historical_r1i1p1f1_gn_185001-201412.yearmean.nc
---- zostoga Omon CMCC-ESM2_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CMCC.CMCC-ESM2.ssp585.r1i1p1f1.Omon.zostoga.gn.v20210126 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_CMCC-ESM2_ssp585_r1i1p1f1_gn_201501-210012.yearmean.nc
---- zostoga Omon CMCC-ESM2_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CMCC.CMCC-ESM2.ssp370.r1i1p1f1.Omon.zostoga.gn.v20210202 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_CMCC-ESM2_ssp370_r1i1p1f1_gn_201501-210012.yearmean.nc
---- zostoga Omon CMCC-ESM2_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CMCC.CMCC-ESM2.ssp245.r1i1p1f1.Omon.zostoga.gn.v20210129 (1 nc file(s))
Input data grid type is generic (points=1)
No need to 

Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_CanESM5-CanOE_ssp245_r1i1p2f1_gn_201501-210012.yearmean.nc
---- zostoga Omon CanESM5-CanOE_r1i1p2f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CCCma.CanESM5-CanOE.ssp126.r1i1p2f1.Omon.zostoga.gn.v20190429 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_CanESM5-CanOE_ssp126_r1i1p2f1_gn_201501-210012.yearmean.nc

---- zostoga Omon CanESM5_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.CCCma.CanESM5.piControl.r1i1p1f1.Omon.zostoga.gn.v20190429 (5 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped 5 existing files, inc zostoga_Omon_CanESM5_piControl_r1i1p1f1_gn_520101-540012.yearmean.nc
---- zostoga Omon CanESM5_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CCCma.CanESM5.historical.r1i1p1f1.Omon.zostoga.gn.v20190429 (1 nc file(s))
Input data grid type is generic (point

Input data grid type is generic (points=1)
No need to regrid
Skipped 86 existing files, inc zostoga_Omon_EC-Earth3-Veg-LR_ssp370_r1i1p1f1_gn_201501-201512.yearmean.nc
---- zostoga Omon EC-Earth3-Veg-LR_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.ssp245.r1i1p1f1.Omon.zostoga.gn.v20201123 (86 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped 86 existing files, inc zostoga_Omon_EC-Earth3-Veg-LR_ssp245_r1i1p1f1_gn_201501-201512.yearmean.nc
---- zostoga Omon EC-Earth3-Veg-LR_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.ssp126.r1i1p1f1.Omon.zostoga.gn.v20201201 (86 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped 86 existing files, inc zostoga_Omon_EC-Earth3-Veg-LR_ssp126_r1i1p1f1_gn_201501-201512.yearmean.nc

---- zostoga Omon EC-Earth3-Veg_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.EC-Earth-Consortium.EC-E

Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_GISS-E2-1-G_historical_r1i1p3f1_gn_185001-201412.yearmean.nc
---- zostoga Omon GISS-E2-1-G_r1i1p3f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp585.r1i1p3f1.Omon.zostoga.gn.v20200115 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_GISS-E2-1-G_ssp585_r1i1p3f1_gn_201501-210012.yearmean.nc
---- zostoga Omon GISS-E2-1-G_r1i1p3f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp370.r1i1p3f1.Omon.zostoga.gn.v20200115 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Skipped existing file: zostoga_Omon_GISS-E2-1-G_ssp370_r1i1p3f1_gn_201501-210012.yearmean.nc
---- zostoga Omon GISS-E2-1-G_r1i1p3f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp245.r1i1p3f1.Omon.zostoga.gn.v20200115 (1 nc file(s))
Input data grid type 

...

Input data grid type is lonlat (points=12960 (144x90))
Trying gencon2
Written hfds_Omon_GISS-E2-1-G_piControl_r1i1p5f1_gn_200001-205012.gencon2_weights.nc
4 input data file(s) to regrid
Written 4 files, inc hfds_Omon_GISS-E2-1-G_piControl_r1i1p5f1_gn_200001-205012.con2.yearmean.nc
---- hfds Omon GISS-E2-1-G_r1i1p5f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.NASA-GISS.GISS-E2-1-G.historical.r1i1p5f1.Omon.hfds.gn.v20190905 (4 nc file(s))
Input data grid type is lonlat (points=12960 (144x90))
Trying gencon2
Written hfds_Omon_GISS-E2-1-G_historical_r1i1p5f1_gn_185001-190012.gencon2_weights.nc
4 input data file(s) to regrid
Written 4 files, inc hfds_Omon_GISS-E2-1-G_historical_r1i1p5f1_gn_185001-190012.con2.yearmean.nc
---- hfds Omon GISS-E2-1-G_r1i1p5f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp585.r1i1p5f1.Omon.hfds.gn.v20200115 (2 nc file(s))
Input data grid type is lonlat (points=12960 (144x90))
Trying gencon2
Written hfds_Omon_GISS-E2-1-G_ssp

Failed to write hfds_Omon_IPSL-CM6A-LR_ssp126_r1i1p1f1_gn_201501-210012.gennn_weights.nc
Trying None

---- hfds Omon KIOST-ESM_r1i1p1f1 piControl -yearmean ----
2 input dirs:
CMIP6.CMIP.KIOST.KIOST-ESM.piControl.r1i1p1f1.Omon.hfds.gr1.v20210319 (2 nc file(s))
CMIP6.CMIP.KIOST.KIOST-ESM.piControl.r1i1p1f1.Omon.hfds.gr1.v20210324 (1 nc file(s))
Using CMIP6.CMIP.KIOST.KIOST-ESM.piControl.r1i1p1f1.Omon.hfds.gr1.v20210324 (most recent) 
Input data grid type is lonlat (points=72000 (360x200))
Trying gencon2
Written hfds_Omon_KIOST-ESM_piControl_r1i1p1f1_gr1_318901-333812.gencon2_weights.nc
1 input data file(s) to regrid
Written hfds_Omon_KIOST-ESM_piControl_r1i1p1f1_gr1_318901-333812.con2.yearmean.nc
---- hfds Omon KIOST-ESM_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.KIOST.KIOST-ESM.historical.r1i1p1f1.Omon.hfds.gr1.v20210412 (1 nc file(s))
Input data grid type is lonlat (points=72000 (360x200))
Trying gencon2
Written hfds_Omon_KIOST-ESM_historical_r1i1p1f1_gr1_185001-201412.

Input data grid type is curvilinear (points=324008 (802x404))
Trying gencon2
Failed to write hfds_Omon_MPI-ESM1-2-HR_ssp585_r1i1p1f1_gn_201501-201912.gencon2_weights.nc
Trying gencon
Failed to write hfds_Omon_MPI-ESM1-2-HR_ssp585_r1i1p1f1_gn_201501-201912.gencon_weights.nc
Trying genbic
Written hfds_Omon_MPI-ESM1-2-HR_ssp585_r1i1p1f1_gn_201501-201912.genbic_weights.nc
18 input data file(s) to regrid
Written 18 files, inc hfds_Omon_MPI-ESM1-2-HR_ssp585_r1i1p1f1_gn_201501-201912.bic.yearmean.nc
---- hfds Omon MPI-ESM1-2-HR_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.DKRZ.MPI-ESM1-2-HR.ssp370.r1i1p1f1.Omon.hfds.gn.v20190710 (18 nc file(s))
Input data grid type is curvilinear (points=324008 (802x404))
Trying gencon2
Failed to write hfds_Omon_MPI-ESM1-2-HR_ssp370_r1i1p1f1_gn_201501-201912.gencon2_weights.nc
Trying gencon
Failed to write hfds_Omon_MPI-ESM1-2-HR_ssp370_r1i1p1f1_gn_201501-201912.gencon_weights.nc
Trying genbic
Written hfds_Omon_MPI-ESM1-2-HR_ssp370_r1i1p1f1_g

Input data grid type is curvilinear (points=130680 (360x363))
Trying gencon2
Written hfds_Omon_MRI-ESM2-0_ssp126_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
2 input data file(s) to regrid
Written 2 files, inc hfds_Omon_MRI-ESM2-0_ssp126_r1i1p1f1_gn_201501-210012.con2.yearmean.nc

---- hfds Omon NESM3_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.NUIST.NESM3.piControl.r1i1p1f1.Omon.hfds.gn.v20190704 (4 nc file(s))
Input data grid type is curvilinear (points=105704 (362x292))
Trying gencon2
Written hfds_Omon_NESM3_piControl_r1i1p1f1_gn_100001-119912.gencon2_weights.nc
4 input data file(s) to regrid
Written 4 files, inc hfds_Omon_NESM3_piControl_r1i1p1f1_gn_100001-119912.con2.yearmean.nc
---- hfds Omon NESM3_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.NUIST.NESM3.historical.r1i1p1f1.Omon.hfds.gn.v20190703 (1 nc file(s))
Input data grid type is curvilinear (points=105704 (362x292))
Trying gencon2
Written hfds_Omon_NESM3_historical_r1i1p1f1_gn_185001-201412.genco

Input data grid type is curvilinear (points=122880 (320x384))
Trying gencon2
Written hfds_Omon_TaiESM1_ssp585_r1i1p1f1_gn_201501-201512.gencon2_weights.nc
86 input data file(s) to regrid
Written 86 files, inc hfds_Omon_TaiESM1_ssp585_r1i1p1f1_gn_201501-201512.con2.yearmean.nc
---- hfds Omon TaiESM1_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.AS-RCEC.TaiESM1.ssp370.r1i1p1f1.Omon.hfds.gn.v20201014 (86 nc file(s))
Input data grid type is curvilinear (points=122880 (320x384))
Trying gencon2
Written hfds_Omon_TaiESM1_ssp370_r1i1p1f1_gn_201501-201512.gencon2_weights.nc
86 input data file(s) to regrid
Written 86 files, inc hfds_Omon_TaiESM1_ssp370_r1i1p1f1_gn_201501-201512.con2.yearmean.nc
---- hfds Omon TaiESM1_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.AS-RCEC.TaiESM1.ssp245.r1i1p1f1.Omon.hfds.gn.v20201125 (86 nc file(s))
Input data grid type is curvilinear (points=122880 (320x384))
Trying gencon2
Written hfds_Omon_TaiESM1_ssp245_r1i1p1f1_gn_201501-20151

### Annual means (yearmean) of regridded Amon data

In [7]:
%%time
# For variables and experiments of interest, apply regridding function to all available source-member pairs
# Loop over variables of interest
table_id = 'Amon'
for variable in ['rlut', 'rsdt', 'rsut', 'tas']:
    # Identify and loop over available source-member pairs
    source_member_pairs = sorted([d.name for d in in_base.glob(f'{variable}/[!.]*_*')])
    for source_member in source_member_pairs:
        # Loop over experiments of interest - do not include piControl for Amon
        for experiment in ['historical', 'ssp585', 'ssp370', 'ssp245', 'ssp126']:
            # Regrid
            temp = regrid_using_cdo(variable=variable,
                                    table_id=table_id,
                                    source_member=source_member,
                                    experiment=experiment,
                                    stat_str='-yearmean',
                                    force=False)
        print('')  # empty line between source-member pairs

---- rlut Amon ACCESS-CM2_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r1i1p1f1.Amon.rlut.gn.v20191108 (1 nc file(s))
Input data grid type is lonlat (points=27648 (192x144))
Trying gencon2
Written rlut_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.con2.yearmean.nc
---- rlut Amon ACCESS-CM2_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp585.r1i1p1f1.Amon.rlut.gn.v20210317 (2 nc file(s))
Input data grid type is lonlat (points=27648 (192x144))
Trying gencon2
Written rlut_Amon_ACCESS-CM2_ssp585_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
2 input data file(s) to regrid
Written 2 files, inc rlut_Amon_ACCESS-CM2_ssp585_r1i1p1f1_gn_201501-210012.con2.yearmean.nc
---- rlut Amon ACCESS-CM2_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp370.r1i

Written rlut_Amon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.con2.yearmean.nc
---- rlut Amon BCC-CSM2-MR_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.BCC.BCC-CSM2-MR.ssp126.r1i1p1f1.Amon.rlut.gn.v20190314 (1 nc file(s))
Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying gencon2
Written rlut_Amon_BCC-CSM2-MR_ssp126_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_BCC-CSM2-MR_ssp126_r1i1p1f1_gn_201501-210012.con2.yearmean.nc

---- rlut Amon BCC-ESM1_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.BCC.BCC-ESM1.historical.r1i1p1f1.Amon.rlut.gn.v20181214 (1 nc file(s))
Input data grid type is gaussian (points=8192 (128x64)  F32)
Trying gencon2
Written rlut_Amon_BCC-ESM1_historical_r1i1p1f1_gn_185001-201412.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_BCC-ESM1_histo

Written 6 files, inc rlut_Amon_CESM2-WACCM_ssp126_r1i1p1f1_gn_201501-206412.con2.yearmean.nc

---- rlut Amon CESM2_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.NCAR.CESM2.historical.r1i1p1f1.Amon.rlut.gn.v20190308 (1 nc file(s))
Input data grid type is lonlat (points=55296 (288x192))
Trying gencon2
Written rlut_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_CESM2_historical_r1i1p1f1_gn_185001-201412.con2.yearmean.nc
---- rlut Amon CESM2_r1i1p1f1 ssp585 -yearmean ----
No suitable input data found
---- rlut Amon CESM2_r1i1p1f1 ssp370 -yearmean ----
No suitable input data found
---- rlut Amon CESM2_r1i1p1f1 ssp245 -yearmean ----
No suitable input data found
---- rlut Amon CESM2_r1i1p1f1 ssp126 -yearmean ----
No suitable input data found

---- rlut Amon CIESM_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.THU.CIESM.historical.r1i1p1f1.Amon.rlut.gr.v20200417 (1 nc file(s))
Input data grid type is 

Written rlut_Amon_CNRM-CM6-1-HR_ssp245_r1i1p1f2_gr_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_CNRM-CM6-1-HR_ssp245_r1i1p1f2_gr_201501-210012.con2.yearmean.nc
---- rlut Amon CNRM-CM6-1-HR_r1i1p1f2 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1-HR.ssp126.r1i1p1f2.Amon.rlut.gr.v20200127 (1 nc file(s))
Input data grid type is gaussian (points=259200 (720x360)  F180)
Trying gencon2
Written rlut_Amon_CNRM-CM6-1-HR_ssp126_r1i1p1f2_gr_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_CNRM-CM6-1-HR_ssp126_r1i1p1f2_gr_201501-210012.con2.yearmean.nc

---- rlut Amon CNRM-CM6-1_r1i1p1f2 historical -yearmean ----
1 input dir: CMIP6.CMIP.CNRM-CERFACS.CNRM-CM6-1.historical.r1i1p1f2.Amon.rlut.gr.v20180917 (1 nc file(s))
Input data grid type is gaussian (points=32768 (256x128)  F64)
Trying gencon2
Written rlut_Amon_CNRM-CM6-1_historical_r1i1p1f2_gr_185001-201412.gencon2_weights.nc
1 input data file(s) to

Input data grid type is gaussian (points=8192 (128x64)  F32)
Trying gencon2
Written rlut_Amon_CanESM5_ssp370_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_CanESM5_ssp370_r1i1p1f1_gn_201501-210012.con2.yearmean.nc
---- rlut Amon CanESM5_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CCCma.CanESM5.ssp245.r1i1p1f1.Amon.rlut.gn.v20190429 (1 nc file(s))
Input data grid type is gaussian (points=8192 (128x64)  F32)
Trying gencon2
Written rlut_Amon_CanESM5_ssp245_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_CanESM5_ssp245_r1i1p1f1_gn_201501-210012.con2.yearmean.nc
---- rlut Amon CanESM5_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CCCma.CanESM5.ssp126.r1i1p1f1.Amon.rlut.gn.v20190429 (2 nc file(s))
Input data grid type is gaussian (points=8192 (128x64)  F32)
Trying gencon2
Written rlut_Amon_CanESM5_ssp126_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
2 input data file(

...

Written rlut_Amon_IITM-ESM_ssp126_r1i1p1f1_gn_201501-202412.gencon2_weights.nc
9 input data file(s) to regrid
Written 9 files, inc rlut_Amon_IITM-ESM_ssp126_r1i1p1f1_gn_201501-202412.con2.yearmean.nc

---- rlut Amon INM-CM4-8_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.INM.INM-CM4-8.historical.r1i1p1f1.Amon.rlut.gr1.v20190530 (3 nc file(s))
Input data grid type is lonlat (points=21600 (180x120))
Trying gencon2
Written rlut_Amon_INM-CM4-8_historical_r1i1p1f1_gr1_185001-194912.gencon2_weights.nc
3 input data file(s) to regrid
Written 3 files, inc rlut_Amon_INM-CM4-8_historical_r1i1p1f1_gr1_185001-194912.con2.yearmean.nc
---- rlut Amon INM-CM4-8_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.INM.INM-CM4-8.ssp585.r1i1p1f1.Amon.rlut.gr1.v20190603 (1 nc file(s))
Input data grid type is lonlat (points=21600 (180x120))
Trying gencon2
Written rlut_Amon_INM-CM4-8_ssp585_r1i1p1f1_gr1_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_IN

Input data grid type is lonlat (points=27648 (192x144))
Trying gencon2
Written rlut_Amon_KACE-1-0-G_historical_r1i1p1f1_gr_185001-201412.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_KACE-1-0-G_historical_r1i1p1f1_gr_185001-201412.con2.yearmean.nc
---- rlut Amon KACE-1-0-G_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NIMS-KMA.KACE-1-0-G.ssp585.r1i1p1f1.Amon.rlut.gr.v20190920 (1 nc file(s))
Input data grid type is lonlat (points=27648 (192x144))
Trying gencon2
Written rlut_Amon_KACE-1-0-G_ssp585_r1i1p1f1_gr_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_KACE-1-0-G_ssp585_r1i1p1f1_gr_201501-210012.con2.yearmean.nc
---- rlut Amon KACE-1-0-G_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NIMS-KMA.KACE-1-0-G.ssp370.r1i1p1f1.Amon.rlut.gr.v20191125 (1 nc file(s))
Input data grid type is lonlat (points=27648 (192x144))
Trying gencon2
Written rlut_Amon_KACE-1-0-G_ssp370_r1i1p1f1_gr_201501-210012.gencon2_wei

Written rlut_Amon_MIROC6_ssp245_r1i1p1f1_gn_201501-210012.con2.yearmean.nc
---- rlut Amon MIROC6_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.MIROC.MIROC6.ssp126.r1i1p1f1.Amon.rlut.gn.v20190627 (1 nc file(s))
Input data grid type is gaussian (points=32768 (256x128)  F64)
Trying gencon2
Written rlut_Amon_MIROC6_ssp126_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_MIROC6_ssp126_r1i1p1f1_gn_201501-210012.con2.yearmean.nc

---- rlut Amon MPI-ESM-1-2-HAM_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.historical.r1i1p1f1.Amon.rlut.gn.v20190627 (9 nc file(s))
Input data grid type is gaussian (points=18432 (192x96)  F48)
Trying gencon2
Written rlut_Amon_MPI-ESM-1-2-HAM_historical_r1i1p1f1_gn_185001-186912.gencon2_weights.nc
9 input data file(s) to regrid
Written 9 files, inc rlut_Amon_MPI-ESM-1-2-HAM_historical_r1i1p1f1_gn_185001-186912.con2.yearmean.nc
---- rlut Amon MPI-ESM-1-2-HAM_

Written rlut_Amon_MRI-ESM2-0_ssp126_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
2 input data file(s) to regrid
Written 2 files, inc rlut_Amon_MRI-ESM2-0_ssp126_r1i1p1f1_gn_201501-210012.con2.yearmean.nc

---- rlut Amon NESM3_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.NUIST.NESM3.historical.r1i1p1f1.Amon.rlut.gn.v20190630 (1 nc file(s))
Input data grid type is gaussian (points=18432 (192x96)  F48)
Trying gencon2
Written rlut_Amon_NESM3_historical_r1i1p1f1_gn_185001-201412.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_NESM3_historical_r1i1p1f1_gn_185001-201412.con2.yearmean.nc
---- rlut Amon NESM3_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NUIST.NESM3.ssp585.r1i1p1f1.Amon.rlut.gn.v20190728 (1 nc file(s))
Input data grid type is gaussian (points=18432 (192x96)  F48)
Trying gencon2
Written rlut_Amon_NESM3_ssp585_r1i1p1f1_gn_201501-210012.gencon2_weights.nc
1 input data file(s) to regrid
Written rlut_Amon_NESM3_ssp585_r1i1p1f1_gn_201

IndexError: list index out of range

In [8]:
! date

Sat Jan 29 01:13:19 +08 2022
